# Adaline and Gradient Descent

In 1960, Bernard Widrow and Ted Hoff published Adaline (Adaptive Linear Neuron), which introduced a key improvement over the perceptron: using gradient descent to minimize a continuous loss function.

In this notebook, we will:
1. Understand the limitation of the perceptron update rule
2. Learn about loss functions and gradient descent
3. Implement Adaline from scratch
4. See why feature scaling matters
5. Compare batch gradient descent vs stochastic gradient descent

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

## The Problem with Perceptron

The perceptron update rule uses the **thresholded output** (0 or 1):

$$w = w + \eta (y - \hat{y}) x$$

This gives us only binary feedback: right or wrong. We never know **how wrong** we are.

- Missing by 0.001? Wrong.
- Missing by 1000? Wrong.

Same update either way.

## Adaline: The Key Insight

Adaline updates weights based on the **continuous net input** instead of the thresholded output.

| | Perceptron | Adaline |
|---|---|---|
| Computes | z = w * x + b | z = w * x + b |
| Updates on | step(z) | z directly |
| Error signal | Binary (0 or 1) | Continuous |

The threshold is only applied for the final prediction, not during learning.

## The Loss Function: Mean Squared Error

To measure "how wrong" we are, we define a **loss function**:

$$L(w, b) = \frac{1}{n} \sum_{i=1}^{n} (y^{(i)} - z^{(i)})^2$$

Where $z^{(i)} = w \cdot x^{(i)} + b$ is the continuous net input.

**Why squared error?**
- Positive and negative errors do not cancel out
- Larger errors are penalized more heavily
- Differentiable everywhere (crucial for gradient descent)

## Gradient Descent

The gradient tells us which direction is "uphill" in weight space. To minimize loss, we move in the **opposite** direction:

$$w = w - \eta \nabla L$$

For MSE loss, the gradient is:

$$\frac{\partial L}{\partial w_j} = -\frac{2}{n} \sum_{i=1}^{n} (y^{(i)} - z^{(i)}) x_j^{(i)}$$

So the update rule becomes:

$$w = w + \eta \frac{2}{n} \sum_{i=1}^{n} (y^{(i)} - z^{(i)}) x^{(i)}$$

## Implementation: AdalineGD

In [ ]:
class AdalineGD:
    """Adaline classifier using batch gradient descent.
    
    Parameters
    ----------
    eta : float
        Learning rate (between 0.0 and 1.0)
    n_iter : int
        Number of passes over the training dataset (epochs)
    random_state : int
        Random seed for weight initialization
    
    Attributes
    ----------
    w_ : 1d-array
        Weights after fitting
    b_ : float
        Bias unit after fitting
    losses_ : list
        MSE loss in each epoch
    """
    
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state
    
    def fit(self, X, y):
        """Fit training data.
        
        Parameters
        ----------
        X : array-like, shape = [n_examples, n_features]
            Training vectors
        y : array-like, shape = [n_examples]
            Target values
        
        Returns
        -------
        self : object
        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=X.shape[1])
        self.b_ = np.float64(0.0)
        self.losses_ = []
        
        for _ in range(self.n_iter):
            net_input = self.net_input(X)
            output = self.activation(net_input)
            errors = y - output
            
            self.w_ += self.eta * 2.0 * X.T.dot(errors) / X.shape[0]
            self.b_ += self.eta * 2.0 * errors.mean()
            
            loss = (errors ** 2).mean()
            self.losses_.append(loss)
        
        return self
    
    def net_input(self, X):
        """Calculate net input: w . x + b"""
        return np.dot(X, self.w_) + self.b_
    
    def activation(self, X):
        """Compute linear activation (identity function)"""
        return X
    
    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.activation(self.net_input(X)) >= 0.5, 1, 0)

## Load the Iris Dataset

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
df = pd.read_csv(url, header=None, encoding='utf-8')

y = df.iloc[0:100, 4].values
y = np.where(y == 'Iris-setosa', 0, 1)

X = df.iloc[0:100, [0, 2]].values

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

## The Learning Rate Problem

Let's see what happens with different learning rates on **unstandardized** data.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

ada1 = AdalineGD(eta=0.1, n_iter=15).fit(X, y)
ax[0].plot(range(1, len(ada1.losses_) + 1), np.log10(ada1.losses_), marker='o')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('log(MSE)')
ax[0].set_title('Adaline - Learning rate 0.1')

ada2 = AdalineGD(eta=0.0001, n_iter=15).fit(X, y)
ax[1].plot(range(1, len(ada2.losses_) + 1), ada2.losses_, marker='o')
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('MSE')
ax[1].set_title('Adaline - Learning rate 0.0001')

plt.tight_layout()
plt.show()

**Too large (0.1):** Loss explodes! We overshoot the minimum and diverge.

**Too small (0.0001):** Loss decreases, but very slowly. Would need many more epochs.

The problem is that our features have very different scales.

## Feature Scaling: Standardization

Standardization transforms each feature to have mean=0 and std=1:

$$x' = \frac{x - \mu}{\sigma}$$

This makes the loss surface more "spherical" so gradient descent can use a consistent step size.

In [ ]:
X_std = np.copy(X)
X_std[:, 0] = (X[:, 0] - X[:, 0].mean()) / X[:, 0].std()
X_std[:, 1] = (X[:, 1] - X[:, 1].mean()) / X[:, 1].std()

print(f"Original X - Feature 0: mean={X[:, 0].mean():.2f}, std={X[:, 0].std():.2f}")
print(f"Original X - Feature 1: mean={X[:, 1].mean():.2f}, std={X[:, 1].std():.2f}")
print()
print(f"Standardized X - Feature 0: mean={X_std[:, 0].mean():.2f}, std={X_std[:, 0].std():.2f}")
print(f"Standardized X - Feature 1: mean={X_std[:, 1].mean():.2f}, std={X_std[:, 1].std():.2f}")

In [ ]:
ada_std = AdalineGD(eta=0.5, n_iter=20).fit(X_std, y)

plt.figure(figsize=(8, 5))
plt.plot(range(1, len(ada_std.losses_) + 1), ada_std.losses_, marker='o')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.title('Adaline with Standardized Features (eta=0.5)')
plt.show()

print(f"Final MSE: {ada_std.losses_[-1]:.4f}")

With standardized features, we can use a much larger learning rate (0.5) and converge quickly.

## Visualizing the Decision Boundary

In [ ]:
def plot_decision_regions(X, y, classifier, resolution=0.02):
    markers = ('o', 's', '^', 'v', '<')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(
        np.arange(x1_min, x1_max, resolution),
        np.arange(x2_min, x2_max, resolution)
    )
    
    lab = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    lab = lab.reshape(xx1.shape)
    
    plt.contourf(xx1, xx2, lab, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(
            x=X[y == cl, 0],
            y=X[y == cl, 1],
            alpha=0.8,
            c=colors[idx],
            marker=markers[idx],
            label=f'Class {cl}',
            edgecolor='black'
        )

In [ ]:
plt.figure(figsize=(8, 6))
plot_decision_regions(X_std, y, classifier=ada_std)
plt.xlabel('Sepal length [standardized]')
plt.ylabel('Petal length [standardized]')
plt.legend(loc='upper left')
plt.title('Adaline Decision Boundary')
plt.show()

In [ ]:
predictions = ada_std.predict(X_std)
accuracy = np.mean(predictions == y)
print(f"Training accuracy: {accuracy * 100:.1f}%")

## Stochastic Gradient Descent (SGD)

Batch gradient descent computes the gradient over **all** samples before updating. This can be slow for large datasets.

**Stochastic Gradient Descent** updates weights after **each** sample:

- Faster iterations
- Noisier updates (can help escape local minima)
- Enables online learning

In [ ]:
class AdalineSGD:
    """Adaline classifier using stochastic gradient descent.
    
    Parameters
    ----------
    eta : float
        Learning rate (between 0.0 and 1.0)
    n_iter : int
        Number of passes over the training dataset (epochs)
    shuffle : bool
        Shuffle training data each epoch to prevent cycles
    random_state : int
        Random seed for weight initialization and shuffling
    """
    
    def __init__(self, eta=0.01, n_iter=10, shuffle=True, random_state=None):
        self.eta = eta
        self.n_iter = n_iter
        self.shuffle = shuffle
        self.random_state = random_state
        self.w_initialized = False
    
    def fit(self, X, y):
        self._initialize_weights(X.shape[1])
        self.losses_ = []
        
        for _ in range(self.n_iter):
            if self.shuffle:
                X, y = self._shuffle(X, y)
            
            losses = []
            for xi, target in zip(X, y):
                losses.append(self._update_weights(xi, target))
            
            avg_loss = np.mean(losses)
            self.losses_.append(avg_loss)
        
        return self
    
    def partial_fit(self, X, y):
        """Fit training data without reinitializing weights (for online learning)."""
        if not self.w_initialized:
            self._initialize_weights(X.shape[1])
        
        if y.ravel().shape[0] > 1:
            for xi, target in zip(X, y):
                self._update_weights(xi, target)
        else:
            self._update_weights(X, y)
        
        return self
    
    def _initialize_weights(self, m):
        self.rgen = np.random.RandomState(self.random_state)
        self.w_ = self.rgen.normal(loc=0.0, scale=0.01, size=m)
        self.b_ = np.float64(0.0)
        self.w_initialized = True
    
    def _shuffle(self, X, y):
        r = self.rgen.permutation(len(y))
        return X[r], y[r]
    
    def _update_weights(self, xi, target):
        output = self.activation(self.net_input(xi))
        error = target - output
        self.w_ += self.eta * 2.0 * xi * error
        self.b_ += self.eta * 2.0 * error
        loss = error ** 2
        return loss
    
    def net_input(self, X):
        return np.dot(X, self.w_) + self.b_
    
    def activation(self, X):
        return X
    
    def predict(self, X):
        return np.where(self.activation(self.net_input(X)) >= 0.5, 1, 0)

In [ ]:
ada_sgd = AdalineSGD(eta=0.01, n_iter=15, random_state=1)
ada_sgd.fit(X_std, y)

plt.figure(figsize=(8, 5))
plt.plot(range(1, len(ada_sgd.losses_) + 1), ada_sgd.losses_, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Average MSE')
plt.title('Adaline SGD Convergence')
plt.show()

print(f"Training accuracy: {np.mean(ada_sgd.predict(X_std) == y) * 100:.1f}%")

## Online Learning with partial_fit

SGD enables **online learning**: updating the model as new data arrives without retraining from scratch.

In [ ]:
ada_online = AdalineSGD(eta=0.01, random_state=1)

for i in range(0, 100, 10):
    X_batch = X_std[i:i+10]
    y_batch = y[i:i+10]
    ada_online.partial_fit(X_batch, y_batch)
    acc = np.mean(ada_online.predict(X_std) == y)
    print(f"After samples {i+1}-{i+10}: accuracy = {acc*100:.1f}%")

## Comparing Batch vs SGD

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

ada_gd = AdalineGD(eta=0.5, n_iter=20).fit(X_std, y)
ax[0].plot(range(1, len(ada_gd.losses_) + 1), ada_gd.losses_, marker='o')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('MSE')
ax[0].set_title('Batch Gradient Descent')

ada_sgd2 = AdalineSGD(eta=0.01, n_iter=20, random_state=1).fit(X_std, y)
ax[1].plot(range(1, len(ada_sgd2.losses_) + 1), ada_sgd2.losses_, marker='o')
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Average MSE')
ax[1].set_title('Stochastic Gradient Descent')

plt.tight_layout()
plt.show()

**Batch GD:** Smooth convergence, but computes gradient over all 100 samples each epoch.

**SGD:** Noisier convergence, but updates 100 times per epoch. Scales better to large datasets.

## Exercises

1. **Learning rate exploration:** Try eta values from 0.001 to 1.0 on standardized data. Plot the loss curves.

2. **Without standardization:** Train Adaline on raw (non-standardized) data. What learning rate works?

3. **Mini-batch SGD:** Modify AdalineSGD to update weights on mini-batches of 16 samples instead of 1.

4. **Compare with Perceptron:** Train both Perceptron and Adaline on the same data. Compare convergence.

In [ ]:
# Exercise 1: Learning rate exploration
# Your code here


In [ ]:
# Exercise 2: Without standardization
# Your code here


In [ ]:
# Exercise 3: Mini-batch SGD
# Your code here


In [ ]:
# Exercise 4: Compare with Perceptron
# Your code here


## Summary

| Concept | What It Does |
|---------|-------------|
| MSE Loss | Measures how wrong we are (continuously) |
| Gradient | Points uphill in weight space |
| Gradient Descent | Steps downhill to minimize loss |
| Learning Rate | Controls step size |
| Standardization | Makes optimization easier |
| SGD | Scales to large datasets |

**The key insight:** By using a continuous loss function and gradient descent, we get smooth optimization that tells us not just whether we are wrong, but **how wrong** and in **which direction** to improve.

This is the foundation of all deep learning. Every neural network, from simple MLPs to GPT, is trained using gradient descent on a loss function.

## Next Steps

In the next session, we will cover:
- **Multi-Layer Perceptrons:** Adding hidden layers to solve non-linear problems
- **Backpropagation:** How gradients flow through multiple layers

## Resources

- [Interactive Demo](https://i33ym.cc/demo-adaline/)
- [Full Essay](https://i33ym.cc/the-adaline/)
- [Slides](https://i33ym.cc/slides-adaline/)